# Explore here

In [42]:
# Import libraries
import pandas as pd
import numpy as np
import sqlite3
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from pickle import dump

In [43]:
### DATA ANALYSIS FOR MOVIE RECOMMENDATION SYSTEM
### Classify 5000 movies based on their credits and predict the 5 movies most similar to the one we enter in the title
# Get dataset for movies
dfm = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv")

In [44]:
# Print the first rows of the dataframe for visualization
dfm.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [45]:
dfm.shape

(4803, 20)

In [46]:
dfm.head().T

,0,1,2,3,4
budget,237000000,300000000,245000000,250000000,260000000
genres,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
homepage,http://www.avatarmovie.com/,http://disney.go.com/disneypictures/pirates/,http://www.sonypictures.com/movies/spectre/,http://www.thedarkknightrises.com/,http://movies.disney.com/john-carter
id,19995,285,206647,49026,49529
keywords,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."
original_language,en,en,en,en,en
original_title,Avatar,Pirates of the Caribbean: At World's End,Spectre,The Dark Knight Rises,John Carter
overview,"In the 22nd century, a paraplegic Marine is di...","Captain Barbossa, long believed to be dead, ha...",A cryptic message from Bond’s past sends him o...,Following the death of District Attorney Harve...,"John Carter is a war-weary, former military ca..."
popularity,150.437577,139.082615,107.376788,112.31295,43.926995
production_companies,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"


In [47]:
# Get dataset for credits
dfc = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv")

In [48]:
# Print the first rows of the dataframe for visualization
dfc.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [49]:
dfc.shape

(4803, 4)

In [50]:
# Check if duplicate rows in the dataframe dfm
print(f"Duplicate rows in dfm: {dfm.duplicated().sum():.2f}")

Duplicate rows in dfm: 0.00


In [51]:
# Check if duplicate rows in the dataframe dfc
print(f"Duplicate rows in dfc: {dfc.duplicated().sum():.2f}")

Duplicate rows in dfc: 0.00


In [52]:
# Count the frequency of unique values in "title" in dataframe dfm
dfm['title'].value_counts()

title
The Host            2
Batman              2
Out of the Blue     2
Awake               1
Beneath Hill 60     1
                   ..
Drag Me to Hell     1
Southpaw            1
Sicario             1
Glory Road          1
30 Days of Night    1
Name: count, Length: 4800, dtype: int64

In [53]:
# Check that there are two rows with the title "The Host" in dataframe dfm
print(dfm.loc[dfm['title'] == 'The Host'])

        budget                                             genres  \
972   44000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
2877  11000000  [{"id": 27, "name": "Horror"}, {"id": 18, "nam...   

                       homepage     id  \
972                         NaN  72710   
2877  http://www.hostmovie.com/   1255   

                                               keywords original_language  \
972   [{"id": 818, "name": "based on novel"}, {"id":...                en   
2877  [{"id": 1261, "name": "river"}, {"id": 1880, "...                ko   

     original_title                                           overview  \
972        The Host  A parasitic alien soul is injected into the bo...   
2877             괴물  Gang-du is a dim-witted man working at his fat...   

      popularity                               production_companies  \
972    42.933027  [{"name": "Nick Wechsler Productions", "id": 8...   
2877   27.655270  [{"name": "Cineclick Asia", "id": 685}, {"name... 

In [54]:
# Count the frequency of unique values in "title" in dataframe dfc
dfc['title'].value_counts()

title
The Host            2
Batman              2
Out of the Blue     2
Awake               1
Beneath Hill 60     1
                   ..
Drag Me to Hell     1
Southpaw            1
Sicario             1
Glory Road          1
30 Days of Night    1
Name: count, Length: 4800, dtype: int64

In [55]:
# Check that there are two rows with the title "The Host" in dataframe dfc
print(dfc.loc[dfc['title'] == 'The Host'])

      movie_id     title                                               cast  \
972      72710  The Host  [{"cast_id": 52, "character": "Melanie Stryder...   
2877      1255  The Host  [{"cast_id": 3, "character": "Park Gang-du", "...   

                                                   crew  
972   [{"credit_id": "52fe487bc3a368484e0fa919", "de...  
2877  [{"credit_id": "52fe42eac3a36847f802ca6b", "de...  


In [56]:
# Eliminate rows with duplicate "title" in dfm and dfc
dfm2 = dfm.drop_duplicates(subset=['title'])
dfc2 = dfc.drop_duplicates(subset=['title'])

In [57]:
dfm2.shape

(4800, 20)

In [58]:
dfc2.shape

(4800, 4)

In [59]:
# Check the lines with at least one isnull value
dfm2.isnull().sum()

budget                     0
genres                     0
homepage                3089
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [60]:
# Fill the nulls with a space in dfm2
dfm3=dfm2.copy()
dfm3 = dfm2.fillna(" ")

In [61]:
# Check the lines with at least one isnull value
dfm3.isnull().sum()

budget                  0
genres                  0
homepage                0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
tagline                 0
title                   0
vote_average            0
vote_count              0
dtype: int64

There are no isnull values in the dataframe dfm3.

In [62]:
# Check the lines with at least one isnull value
dfc2.isnull().sum()

movie_id    0
title       0
cast        0
crew        0
dtype: int64

There are no isnull values in the dataframe dfc2.

In [63]:
# Connect to sqlite3 database 
conn = sqlite3.connect("../data/movies_database.db")
# Store Dataframes in separate tables
dfm3.to_sql("movies_table", conn, if_exists = "replace", index = False)
dfc2.to_sql("credits_table", conn, if_exists = "replace", index = False)

4800

In [64]:
# Join the two tables with SQL
join_tables = """
    SELECT *
    FROM movies_table
    INNER JOIN credits_table
    ON movies_table.title = credits_table.title;
"""
df_total = pd.read_sql_query(join_tables, conn)
conn.close()
df_total.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,title,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [65]:
df_total.head().T

,0,1,2,3,4
budget,237000000,300000000,245000000,250000000,260000000
genres,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
homepage,http://www.avatarmovie.com/,http://disney.go.com/disneypictures/pirates/,http://www.sonypictures.com/movies/spectre/,http://www.thedarkknightrises.com/,http://movies.disney.com/john-carter
id,19995,285,206647,49026,49529
keywords,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."
original_language,en,en,en,en,en
original_title,Avatar,Pirates of the Caribbean: At World's End,Spectre,The Dark Knight Rises,John Carter
overview,"In the 22nd century, a paraplegic Marine is di...","Captain Barbossa, long believed to be dead, ha...",A cryptic message from Bond’s past sends him o...,Following the death of District Attorney Harve...,"John Carter is a war-weary, former military ca..."
popularity,150.437577,139.082615,107.376788,112.31295,43.926995
production_companies,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"


In [66]:
df_total.shape

(4800, 24)

In [67]:
# Remove duplicate columns in the dataframe
df_total = df_total.loc[:,~df_total.columns.duplicated()]
df_total.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [68]:
# Check if duplicates
print(f"Duplicate values in dataframe: {df_total.duplicated().sum():.2f}")

Duplicate values in dataframe: 0.00


There are no duplicated values in the dataframe.

In [69]:
# Select the following columns in the dataframe: movie_id, title, overview, genres, keywords, cast, crew
dfmovies = df_total[["movie_id", "title", "overview", "genres", "keywords", "cast", "crew"]]

In [70]:
dfmovies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [71]:
print(type(dfmovies['genres']))

<class 'pandas.core.series.Series'>


In [72]:
print(dfmovies['genres'])

0       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1       [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3       [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
                              ...                        
4795    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4796    [{"id": 35, "name": "Comedy"}, {"id": 10749, "...
4797    [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...
4798                                                   []
4799                  [{"id": 99, "name": "Documentary"}]
Name: genres, Length: 4800, dtype: object


In [73]:
# Transform the columns overview, genres, keywords, cast, crew
dfmovies["overview"] = dfmovies["overview"].apply(lambda x: [x])
dfmovies["genres"] = dfmovies["genres"].apply(lambda x: [item["name"] for item in json.loads(x)])
dfmovies["keywords"] = dfmovies["keywords"].apply(lambda x: [item["name"] for item in json.loads(x)])
dfmovies["cast"] = dfmovies["cast"].apply(lambda x: [item["name"] for item in json.loads(x)][:3])
dfmovies["crew"] = dfmovies["crew"].apply(lambda x: " ".join([crew_member["name"] for crew_member in json.loads(x) if crew_member["job"] == "Director"]))
dfmovies.head()

/tmp/ipykernel_26730/1813665312.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmovies["overview"] = dfmovies["overview"].apply(lambda x: [x])
/tmp/ipykernel_26730/1813665312.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmovies["genres"] = dfmovies["genres"].apply(lambda x: [item["name"] for item in json.loads(x)])
/tmp/ipykernel_26730/1813665312.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,206647,Spectre,[A cryptic message from Bond’s past sends him ...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,49026,The Dark Knight Rises,[Following the death of District Attorney Harv...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,49529,John Carter,"[John Carter is a war-weary, former military c...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton


In [74]:
# Convert the colums overview, genres, keywords, cast, crew to string
dfmovies["overview"] = dfmovies["overview"].apply(lambda x: [str(x)])
dfmovies["genres"] = dfmovies["genres"].apply(lambda x: [str(genre) for genre in x])
dfmovies["keywords"] = dfmovies["keywords"].apply(lambda x: [str(keyword) for keyword in x])
dfmovies["cast"] = dfmovies["cast"].apply(lambda x: [str(actor) for actor in x])
dfmovies["crew"] = dfmovies["crew"].apply(lambda x: [str(crew_member) for crew_member in x])
# Create a column "tags" combining the columns overview, genres, keywords, cast and crew
dfmovies["tags"] = dfmovies["overview"] + dfmovies["genres"] + dfmovies["keywords"] + dfmovies["cast"] + dfmovies["crew"]

# Remove the spaces between the words for the column tags
dfmovies["tags"] = dfmovies["tags"].apply(lambda x: ",".join(x).replace(",", " "))

/tmp/ipykernel_26730/4007420268.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmovies["overview"] = dfmovies["overview"].apply(lambda x: [str(x)])
/tmp/ipykernel_26730/4007420268.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmovies["genres"] = dfmovies["genres"].apply(lambda x: [str(genre) for genre in x])
/tmp/ipykernel_26730/4007420268.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [75]:
# Drop the columns movie_id, overview, genres, keywords, cast, crew in the dataframe
dfmovies.drop(columns = ["movie_id", "genres", "keywords", "cast", "crew", "overview"], inplace = True)
dfmovies.head()

/tmp/ipykernel_26730/2966319436.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmovies.drop(columns = ["movie_id", "genres", "keywords", "cast", "crew", "overview"], inplace = True)


,title,tags
0,Avatar,['In the 22nd century a paraplegic Marine is ...
1,Pirates of the Caribbean: At World's End,['Captain Barbossa long believed to be dead ...
2,Spectre,['A cryptic message from Bond’s past sends him...
3,The Dark Knight Rises,"[""Following the death of District Attorney Har..."
4,John Carter,"[""John Carter is a war-weary former military ..."


In [76]:
# Vectorize the "tags" column for all the movies
vectorizer = CountVectorizer(stop_words='english')
tags_vec = vectorizer.fit_transform(dfmovies["tags"])


In [77]:
tags_vec.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4800, 26960))

In [78]:
vectorizer.get_feature_names_out()[300:400]

array(['70mm', '70s', '72', '73', '747', '74th', '75', '75th', '78', '79',
       '7th', '80', '800', '80s', '81', '83', '84', '86', '88', '8km',
       '8th', '90', '9000', '900lb', '90210', '90s', '911', '922', '92nd',
       '93', '95', '96', '98', '99', '9th', '_wizard', 'aa', 'aaa',
       'aaliyah', 'aames', 'aamir', 'aang', 'aaran', 'aaron', 'aarons',
       'aasheekaa', 'aasif', 'aba', 'abaddon', 'abagnale', 'abai',
       'abandon', 'abandoned', 'abandoning', 'abandonment', 'abandons',
       'abba', 'abbas', 'abbass', 'abbate', 'abberline', 'abbey', 'abbie',
       'abbott', 'abby', 'abdalla', 'abdicates', 'abdication', 'abduct',
       'abducted', 'abduction', 'abductions', 'abductors', 'abducts',
       'abdul', 'abdullah', 'abe', 'abel', 'abell', 'aberdeen',
       'abernathy', 'abhinay', 'abhishek', 'abhorrent', 'abhors', 'abi',
       'abide', 'abigail', 'abilities', 'ability', 'abkarian', 'ablai',
       'able', 'ableman', 'abo', 'aboard', 'abode', 'abolish',
       'ab

In [79]:
vectorizer.vocabulary_

{'22nd': 226,
 'century': 4188,
 'paraplegic': 17739,
 'marine': 15059,
 'dispatched': 7023,
 'moon': 16103,
 'pandora': 17682,
 'unique': 25254,
 'mission': 15905,
 'torn': 24506,
 'following': 9331,
 'orders': 17338,
 'protecting': 19067,
 'alien': 916,
 'civilization': 4627,
 'action': 523,
 'adventure': 654,
 'fantasy': 8779,
 'science': 21277,
 'fiction': 9015,
 'culture': 5913,
 'clash': 4662,
 'future': 9705,
 'space': 22591,
 'war': 26019,
 'colony': 4959,
 'society': 22436,
 'travel': 24714,
 'futuristic': 9707,
 'romance': 20621,
 'tribe': 24777,
 'planet': 18401,
 'cgi': 4205,
 'soldier': 22468,
 'battle': 2261,
 'love': 14533,
 'affair': 684,
 'anti': 1250,
 'power': 18711,
 'relations': 19942,
 'mind': 15797,
 'soul': 22562,
 '3d': 259,
 'sam': 20989,
 'worthington': 26594,
 'zoe': 26904,
 'saldana': 20959,
 'sigourney': 22019,
 'weaver': 26156,
 'captain': 3863,
 'barbossa': 2134,
 'long': 14450,
 'believed': 2439,
 'dead': 6225,
 'come': 4998,
 'life': 14198,
 'headed': 

In [80]:
# NearestNeighbors model for vectorized "tags"
neigh_model = NearestNeighbors(n_neighbors=6, metric='cosine')
neigh_model.fit(tags_vec)


NearestNeighbors(metric='cosine', n_neighbors=6)

In [81]:
# Select the title and the tags of the movie 2758
print(dfmovies.iloc[2758,])

title                                 The Astronaut Farmer
tags     ["Texan Charles Farmer left the Air Force as a...
Name: 2758, dtype: object


In [82]:
# Get the index of the movie "The Astronaut Farmer"
movie1="The Astronaut Farmer"
movie1_index = dfmovies[dfmovies["title"] == movie1].index[0]
movie1_index

np.int64(2758)

In [83]:
# Get the tags of the movie "The Astronaut Farmer"
tag1=dfmovies.at[movie1_index, 'tags']
tag1

'["Texan Charles Farmer left the Air Force as a young man to save the family ranch when his dad died. Like most American ranchers  he owes his bank. Unlike most  he\'s an astrophysicist with a rocket in his barn - one he\'s built and wants to take into space. It\'s his dream. The FBI puts him under surveillance when he tries to buy rocket fuel  and the FAA stalls him when he files a flight plan – but Charles is undeterred."] Adventure Comedy Drama Science Fiction nasa texas dream fbi bankrupt peasant earth farm insanity rocket life\'s dream spinner space independent film astronaut family Billy Bob Thornton Virginia Madsen Max Thieriot M i c h a e l   P o l i s h'

In [84]:
# Vectorize the tags of the movie "The Astronaut Farmer"
test_vectors = vectorizer.transform(["tag1"])
test_vectors.toarray()

array([[0, 0, 0, ..., 0, 0, 0]], shape=(1, 26960))

In [85]:
# Definition of the function to recommend 5 movies based on the tags of a movie
def recommend(movie):
    # Select the index of the movie
    movie_index = dfmovies[dfmovies["title"] == movie].index[0]
    # Select the tags of the movie
    tags_movie=dfmovies.at[movie_index, 'tags']
    # Vectorize the tags of the movie
    tags_vec_movie = (vectorizer.transform([tags_movie]))
    # Calculate the 5 Nearest Neighbourgs and itself
    my_neigh=neigh_model.kneighbors(tags_vec_movie,6,return_distance=False)
    # Choose the 5 Nearest Neighbourgs
    print(f"Recommended films for {movie}:")
    i = 0
    while i<5:
        i = i+1
        print(dfmovies.iloc[my_neigh[0,i]]["title"])
    print()

recommend("Avatar")

recommend("The Astronaut Farmer")

recommend("George and the Dragon")

Recommended films for Avatar:
Aliens
Mission to Mars
Moonraker
Alien³
Alien

Recommended films for The Astronaut Farmer:
Nicholas Nickleby
Eulogy
Space Dogs
Gravity
Lost in Space

Recommended films for George and the Dragon:
The Hobbit: The Desolation of Smaug
Dragon Nest: Warriors' Dawn
Dragon Hunters
Eragon
How to Train Your Dragon 2



In [86]:
# Save the NearestNeighbors model
dump(neigh_model, open("../models/nearest_neighbors-6_metric-cosine.sav", "wb"))